In [1]:
import os
work_dir = "/content/tensorflow-tutorial/"
if os.getcwd() != work_dir:
    !git clone https://github.com/sewerynzalupka/tensorflow-tutorial.git
os.chdir(work_dir)

!pip install -r requirements.txt

Cloning into 'tensorflow-tutorial'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (489/489), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 489 (delta 233), reused 477 (delta 223), pack-reused 0
Receiving objects: 100% (489/489), 62.70 MiB | 22.67 MiB/s, done.
Resolving deltas: 100% (233/233), done.
    100% |████████████████████████████████| 337kB 6.4MB/s 
ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.15
    Uninstalling prompt-toolkit-1.0.15:
      Successfully uninstalled prompt-toolkit-1.0.15


# Imports

In [2]:
%matplotlib inline
import PIL
import os
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


# Dataset - SigComp2011

Download and extract training and test sets (or use cached files)

In [3]:
import signatures

signatures.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.
- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [25]:
!ls data/sigComp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet

'Offline Forgeries'  'Offline Genuine'


This dataset has different directory structure than the Keras API requires, so copy the files into separate directories for the training- and test-sets.

In [54]:
from PIL import Image
import pickle
import cv2

def NormalizedImage(img, new_height, new_width):
  cImg = 255*np.ones((new_height,new_width,3))
  width =  np.size(img,1)
  height = np.size(img,0)
  left = int((new_width - width)/2.)
  top = int(((new_height - height)/2.))
  right = int(((width + new_width)/2.))
  bottom = int(((height + new_height)/2.))
  cImg[top:bottom, left:right,:] = img
  return cImg

path = os.listdir('data/sigComp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/')
path.sort(key=str.lower)

print(path[0], path[0][:3])

def get_filenames(ext, dir):
  filenames = []
  for root, dirs, files in os.walk(dir):
    for file in files:
      if file.lower().endswith(ext.lower()):
        filenames.append(file)
  return filenames

print(get_filenames('.png', 'data/sigComp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/Offline Genuine/'))

import re
forgery_filename_pattern = r"(?P<forger>\d{4})(?P<signer>\d{3})_(?P<i>\d{2})"
genuine_filename_pattern = r"(?P<signer>\d{3})_(?P<i>\d{2})"

signers = set()
for filename in get_filenames('.png', 'data/sigComp2011/trainingSet/OfflineSignatures/Dutch/'):
#   forgery_match = re.match(forgery_filename_pattern, filename)
  genuine_match = re.match(genuine_filename_pattern, filename)
#   if forgery_match:
#     signers.add(forgery_match.group('signer'))
  if genuine_match:
    signers.add(genuine_match.group('signer'))
    
print(sorted(list(signers)))

001_01.PNG 001
['016_12.PNG', '014_17.PNG', '003_04.PNG', '002_20.PNG', '003_10.PNG', '001_13.PNG', '002_12.PNG', '014_16.PNG', '002_16.PNG', '001_05.PNG', '001_20.PNG', '015_05.PNG', '006_14.PNG', '009_11.PNG', '006_08.PNG', '009_05.PNG', '012_12.PNG', '009_22.PNG', '002_21.PNG', '003_18.PNG', '012_05.PNG', '014_04.PNG', '009_24.PNG', '016_16.PNG', '001_15.PNG', '015_08.PNG', '016_02.PNG', '004_22.PNG', '006_01.PNG', '015_11.PNG', '006_11.PNG', '004_21.PNG', '004_13.PNG', '009_20.PNG', '006_23.PNG', '015_07.PNG', '014_02.PNG', '009_15.PNG', '012_20.PNG', '006_21.PNG', '001_14.PNG', '004_19.PNG', '009_04.PNG', '016_05.PNG', '003_08.PNG', '016_07.PNG', '003_20.PNG', '009_21.PNG', '002_18.PNG', '014_07.PNG', '004_08.PNG', '001_08.PNG', '014_18.PNG', '012_11.PNG', '016_06.PNG', '001_04.PNG', '003_17.PNG', '006_17.PNG', '012_13.PNG', '014_21.PNG', '003_13.PNG', '016_01.PNG', '012_09.PNG', '009_14.PNG', '001_17.PNG', '003_16.PNG', '004_23.PNG', '014_11.PNG', '006_20.PNG', '016_13.PNG', '016

In [40]:
class SignatureVerificationDataset:
    def __init__(self, train_dir, test_dir, exts='.png'):
        """
        Create a data-set consisting of the filenames in the given directory
        and sub-dirs that match the given filename-extensions.

        :param train_dir:
            Root-dir for the files in the data-set.
            This would be 'knifey-spoony/' in the example above.
            
        :param test_dir:
            Root-dir for the files in the data-set.
            This would be 'knifey-spoony/' in the example above.

        :param exts:
            String or tuple of strings with valid filename-extensions.
            Not case-sensitive.

        :return:
            Object instance.
        """

        # Extend input directories to the full path.
        train_dir = os.path.abspath(train_dir)
        test_dir = os.path.abspath(test_dir)

        # Input directories.
        self.train_dir = train_dir
        self.test_dir = test_dir

        # Convert all file-extensions to lower-case.
        self.exts = tuple(ext.lower() for ext in exts)

        # Names for the classes.
        self.class_names = []

        # Filenames for all the files in the training-set.
        self.filenames = []

        # Filenames for all the files in the test-set.
        self.filenames_test = []

        # Class-number for each file in the training-set.
        self.class_numbers = []

        # Class-number for each file in the test-set.
        self.class_numbers_test = []

        # Total number of classes in the data-set.
        self.num_classes = 0

        # For all files/dirs in the input directory.
        for name in os.listdir(in_dir):
            # Full path for the file / dir.
            current_dir = os.path.join(in_dir, name)

            # If it is a directory.
            if os.path.isdir(current_dir):
                # Add the dir-name to the list of class-names.
                self.class_names.append(name)

                # Training-set.

                # Get all the valid filenames in the dir (not sub-dirs).
                filenames = self._get_filenames(current_dir)

                # Append them to the list of all filenames for the training-set.
                self.filenames.extend(filenames)

                # The class-number for this class.
                class_number = self.num_classes

                # Create an array of class-numbers.
                class_numbers = [class_number] * len(filenames)

                # Append them to the list of all class-numbers for the training-set.
                self.class_numbers.extend(class_numbers)

                # Test-set.

                # Get all the valid filenames in the sub-dir named 'test'.
                filenames_test = self._get_filenames(os.path.join(current_dir, 'test'))

                # Append them to the list of all filenames for the test-set.
                self.filenames_test.extend(filenames_test)

                # Create an array of class-numbers.
                class_numbers = [class_number] * len(filenames_test)

                # Append them to the list of all class-numbers for the test-set.
                self.class_numbers_test.extend(class_numbers)

                # Increase the total number of classes in the data-set.
                self.num_classes += 1
                
    def _get_filenames(self, dir):
        """
        Create and return a list of filenames with matching extensions in the given directory.

        :param dir:
            Directory to scan for files. Sub-dirs are not scanned.

        :return:
            List of filenames. Only filenames. Does not include the directory.
        """
        filenames = []
        for root, dirs, files in os.walk(dir):
          for file in files:
            if file.lower().endswith(self.exts):
              filenames.append(os.path.join(root, file))
        return filenames


ds = SignatureVerificationDataset(test_dir='data/sigComp2011/Testdata_SigComp2011/SigComp11-Offlinetestset/Dutch/', train_dir='data/sigComp2011/trainingSet/OfflineSignatures/Dutch/TrainingSet/')
print(ds.class_names, ds.filenames, ds.num_classes)
# base_model = InceptionV3(include_top = False, weights='imagenet', input_shape= (551,1117,3))

['Offline Genuine', 'Offline Forgeries'] ['016_12.PNG', '014_17.PNG', '003_04.PNG', '002_20.PNG', '003_10.PNG', '001_13.PNG', '002_12.PNG', '014_16.PNG', '002_16.PNG', '001_05.PNG', '001_20.PNG', '015_05.PNG', '006_14.PNG', '009_11.PNG', '006_08.PNG', '009_05.PNG', '012_12.PNG', '009_22.PNG', '002_21.PNG', '003_18.PNG', '012_05.PNG', '014_04.PNG', '009_24.PNG', '016_16.PNG', '001_15.PNG', '015_08.PNG', '016_02.PNG', '004_22.PNG', '006_01.PNG', '015_11.PNG', '006_11.PNG', '004_21.PNG', '004_13.PNG', '009_20.PNG', '006_23.PNG', '015_07.PNG', '014_02.PNG', '009_15.PNG', '012_20.PNG', '006_21.PNG', '001_14.PNG', '004_19.PNG', '009_04.PNG', '016_05.PNG', '003_08.PNG', '016_07.PNG', '003_20.PNG', '009_21.PNG', '002_18.PNG', '014_07.PNG', '004_08.PNG', '001_08.PNG', '014_18.PNG', '012_11.PNG', '016_06.PNG', '001_04.PNG', '003_17.PNG', '006_17.PNG', '012_13.PNG', '014_21.PNG', '003_13.PNG', '016_01.PNG', '012_09.PNG', '009_14.PNG', '001_17.PNG', '003_16.PNG', '004_23.PNG', '014_11.PNG', '006_2